install library

In [2]:
! pip install Backtesting

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=2ca1e0b6ad8988a144ba294d17f3bb8553799b326c239f5cc5a252b247502edb
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built Backtesting
--2024-01-02 12:22:03--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-01-02 12:22:03--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloa

loading the data and data preprocessing




In [3]:
import fileinput
import sys

# 文件路徑
file_path = '/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py'

# 需要修改的行號
line_to_edit = 148

# 新的內容
new_content = "                new_bar_idx = new_index.get_indexer([mean_time], method='nearest')[0]"

# 安全檢查：備份原始文件
backup_file_path = file_path + '.backup'
with open(file_path, 'r') as original_file, open(backup_file_path, 'w') as backup_file:
    backup_file.write(original_file.read())

# 嘗試修改文件
try:
    with fileinput.FileInput(file_path, inplace=True, backup='.bak') as file:
        for i, line in enumerate(file, start=1):
            if i == line_to_edit:
                print(new_content)  # 替換為新內容
            else:
                print(line, end='')  # 保留原始內容

    print("文件修改成功")
except Exception as e:
    print(f"發生錯誤: {e}", file=sys.stderr)
    # 發生錯誤時恢復原始文件
    with open(backup_file_path, 'r') as backup_file, open(file_path, 'w') as original_file:
        original_file.write(backup_file.read())
    print("文件已恢復到原始狀態。", file=sys.stderr)

文件修改成功


In [4]:
# 掛載自己的雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import pandas as pd

# 請更換成自己檔案在雲端上的路徑
data_path = "/content/drive/MyDrive/EURUSD_M1_train.csv"

#匯入資料
df = pd.read_csv(data_path, names=GOOG.columns)
df.index = pd.to_datetime(df.index)
df.drop(index=df.index[0], inplace=True)
df = df.astype("float")

df.head()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
<ipython-input-5-930e749d39a7>:10: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, names=GOOG.columns)


,Open,High,Low,Close,Volume
2017-01-02 02:00:00,1.05155,1.05197,1.05155,1.05190,0.0
2017-01-02 02:01:00,1.05209,1.05209,1.05177,1.05179,0.0
2017-01-02 02:02:00,1.05177,1.05198,1.05177,1.05178,0.0
2017-01-02 02:03:00,1.05188,1.05200,1.05188,1.05200,0.0
2017-01-02 02:04:00,1.05196,1.05204,1.05196,1.05203,0.0


backtesting strategy

In [6]:
import talib
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import numpy as np

class OptimizedStrategy(Strategy):

      #short_window = 45
      #long_window = 62
      #stoch_k_period = 5
      #stoch_d_period = 3
      rsi_period = 11#14
      rsi_oversold = 40#30
      rsi_overbought = 83#70
      #stoch_oversold = 20
      #stoch_overbought = 80
      macd_short_period = 11#12
      macd_long_period = 35#26
      macd_signal_period = 9#9


      def init(self):
          close = self.data.Close  # 獲取收盤價數據
          #self.short_ma = self.I(talib.EMA, close, self.short_window)
          #self.long_ma = self.I(talib.EMA, close, self.long_window)
          #self.stoch_k, self.stoch_d = self.I(talib.STOCH, self.data.High, self.data.Low, close, self.stoch_k_period, self.stoch_d_period, self.stoch_d_period)
          self.rsi = self.I(talib.RSI, close, self.rsi_period)
          #self.atr = self.I(talib.RSI, close, atr_period)
          #self.macd, self.signal, self.hist = talib.MACD(self.data.Close)
          self.macd, self.signal, self.hist = self.I(talib.MACD, close, self.macd_short_period, self.macd_long_period, self.macd_signal_period)
          self.upper_band, self.middle_band, self.lower_band = talib.BBANDS(close, 10, 2, 2)


      def next(self):
          position_size = int(self.equity/self.data.Close[-1])
          if self.rsi[-1] < self.rsi_oversold and self.hist[-1]>0 and self.data.Close[-1] < self.lower_band[-1]:
              self.buy(size=position_size)
          elif self.rsi[-1] > self.rsi_overbought and self.hist[-1]<0 and self.data.Close[-1] > self.upper_band[-1]:
              self.sell(size=position_size)


bt = Backtest(df, OptimizedStrategy, cash=100000000, commission = 0.00005)

stats = bt.run()

print(stats)

<ipython-input-6-24bfeeca1694>:43: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, OptimizedStrategy, cash=100000000, commission = 0.00005)


Start                     2017-01-02 02:00:00
End                       2022-12-30 16:58:00
Duration                   2188 days 14:58:00
Exposure Time [%]                   99.980148
Equity Final [$]             132956842.774846
Equity Peak [$]              133175101.834326
Return [%]                          32.956843
Buy & Hold Return [%]                1.772032
Return (Ann.) [%]                    3.895633
Volatility (Ann.) [%]                3.449552
Sharpe Ratio                         1.129316
Sortino Ratio                        1.968533
Calmar Ratio                         1.038283
Max. Drawdown [%]                   -3.751996
Avg. Drawdown [%]                   -0.117821
Max. Drawdown Duration      975 days 19:30:00
Avg. Drawdown Duration        2 days 02:27:00
# Trades                                    6
Win Rate [%]                            100.0
Best Trade [%]                      11.723222
Worst Trade [%]                      0.140024
Avg. Trade [%]                    

In [10]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '8H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:

GridPlot(id='p2325', ...)

parameter optimization

In [8]:
import optuna
import talib
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import numpy as np

def objective(trial):
    # 傳遞優化參數給策略
    #short_window = trial.suggest_int('short_window', 5, 10)
    #long_window = trial.suggest_int('long_window', 14, 50)
    short_window_rsi = trial.suggest_int('short_window rsi', 5, 20)
    #long_window_rsi = trial.suggest_int('long_window rsi', 14, 50)
    #stoch_period = trial.suggest_int('stoch_period', 6, 20)
    #toch_k_period = trial.suggest_int('stoch k period', 3, 10)
    #stoch_d_period = trial.suggest_int('stoch k period', 3, 10)
    #stoch_overbought = trial.suggest_int('stoch overbought', 70, 90)
    #stoch_oversold = trial.suggest_int('stoch oversold', 10, 30)
    #atr_period = trial.suggest_int('atr_period', 6, 20)
    #rsi_period = trial.suggest_int('rsi_period', 6, 20)
    rsi_oversold = trial.suggest_int('rsi_oversold', 30, 50)
    rsi_overbought = trial.suggest_int('rsi_overbought', 80, 90)
    macd_short_period = trial.suggest_int('macd short period', 8, 15)
    macd_long_period = trial.suggest_int('macd long period', 21, 50)
    macd_signal_period = trial.suggest_int('macd signal period', 8, 15)
    #time_length = trial.suggest_int('time length', 5, 50)
    #lookback = trial.suggest_int('lookback', 5, 20)

    #risk_factor = trial.suggest_int('risk factor', 0.01, 0.9)
    #atr_factor = trial.suggest_int('atr factor', 1, 20)
    #TPSLRatio = trial.suggest_int('TPSLRatio', 1.2, 3)


    class OptimizedStrategy(Strategy):
        def init(self):
            close = self.data.Close  # 獲取收盤價數據
            #self.short_ma = self.I(SMA, close, short_window)
            #self.long_ma = self.I(SMA, close, long_window)
            #self.rsi_l = self.I(talib.RSI, close, long_window_rsi)
            self.rsi = self.I(talib.RSI, close, short_window_rsi)
            #self.stoch_k, self.stoch_d = self.I(talib.STOCHRSI, self.data.Close, timeperiod=9, fastk_period=6, fastd_period=6)
            #self.atr = self.I(talib.ATR, self.data.High, self.data.Low, close, atr_period)
            #self.macd, self.signal, self.hist = talib.MACD(self.data.Close)
            self.macd, self.signal, self.hist = self.I(talib.MACD, close, macd_short_period,  macd_long_period, macd_signal_period)
            self.upper_band, self.middle_band, self.lower_band = talib.BBANDS(close, 10, 2, 2)


        def next(self):
            position_size = int(self.equity/self.data.Close[-1])
            '''if self.data.Close[-1] < self.lower_band[-1] and self.hist[-1]>0 and self.data.Low[-1] < min(self.data.Low[-lookback:-1]) and self.hist[-1] > max(self.hist[-lookback:-1]) and self.rsi[-1] < rsi_oversold:
                self.buy(size=position_size)
            elif self.data.Close[-1] > self.upper_band[-1] and self.hist[-1]<0 and self.data.High[-1] > max(self.data.High[-lookback:-1]) and self.hist[-1] < min(self.hist[-lookback:-1]) and self.rsi[-1] > rsi_overbought:
                self.sell(size=position_size)'''
            if self.data.Close[-1] < self.lower_band[-1] and  self.rsi[-1] < rsi_oversold and self.hist[-1]>0:
                self.buy(size=position_size)
            elif self.data.Close[-1] > self.upper_band[-1] and self.rsi[-1] > rsi_overbought and self.hist[-1]<0:
                self.sell(size=position_size)


    # 設定回測
    bt = Backtest(df, OptimizedStrategy, cash=100000000, commission = 0.00005)
    stats = bt.run()

    # 這裡返回 Sharpe 比率作為優化目標
    return stats['Sharpe Ratio']


# 開始優化
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2000)

# 打印最佳參數組合
best_params = study.best_params
print(f"Best Parameters: {best_params}")

[I 2024-01-02 12:28:06,587] A new study created in memory with name: no-name-947d25da-c0ce-431e-ac78-e95b3f1af7fd
<ipython-input-8-bc10a3611ae2>:61: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, OptimizedStrategy, cash=100000000, commission = 0.00005)
[I 2024-01-02 12:29:02,504] Trial 0 finished with value: 0.5050442821037184 and parameters: {'short_window rsi': 7, 'rsi_oversold': 50, 'rsi_overbought': 89, 'macd short period': 8, 'macd long period': 25, 'macd signal period': 15}. Best is trial 0 with value: 0.5050442821037184.
<ipython-input-8-bc10a3611ae2>:61: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df, OptimizedStrategy, cash=100000000, commission = 0.00005)
[I 2024-01-02 12:29:51,059] Trial 1 finished with value: 0.04197581884464151 and parameters: {'short_window rsi': 19, 'rsi_oversold': 46, 'rsi_overbought': 85, 'macd short period': 15, 'macd long period': 42, 'macd signal period': 14}. Best is tri

KeyboardInterrupt: ignored